### This notebook contains code for the processing of raw MODIS geotiff files as used and described in Lang et al. (xxxx)

Inputs: 
1) MODIS fSCA (from MODSCAG) and RF (from MODDRFS) daily.tif files in default sinusoidal projection.
2) Shapefile for watershed/region of interest.

Outputs:
1) Step - Folder with MODIS data clipped by watershed.
2) Step - Folder with MODIS data clipped by watershed and reprojected.
3) Step - Folder with binary snow cover fraction mask rasters.
4) Step - Dataset containing data from clipped and reprojected RF, fSCA, and mask rasters.
5) Step - Same dataset as Step 4, but also containing RF masked by snow cover fraction.
6) Step (final output) - Same dataset as Step 5, but containing RF masked by snow cover fraction and snow persistence.

Note: steps 1-2 must be completed for both RF and fSCA data seperately. Steps 4-6 must be completed in order and rely on outputs from steps 1-3.


###### Author: Otto Lang. Assited by: Patrick Naple and Dillon Ragar.
###### Contact otto.lang@utah.edu for questions.

#### 1. Clipping 

In [ ]:
#Clipping daily MODIS Geotiffs to the watershed of interest 
#Creates daily clipped geotiffs and stores them in an output directory

#Loading packages for this step
import gdal
import fnmatch
import os
gdal.UseExceptions()

#Directories (update this)
inDir = "/Path/to/folder/with/MODIS/geotiffs"
outDir = "/Path/to/clipped/output/folder"
shpin = "/Path/to/watershed/shapefile.shp"
os.chdir (inDir)

#Defining findRasters function
def findRasters (path, filter):
    for root, dirs, files in os.walk(path, filter):
        for file in fnmatch.filter(files, filter):
            yield os.path.join (root, file)

#Creating new rasters in outDir directory
for raster in findRasters (inDir, '*.tif'):
    (infilepath, infilename)= os.path.split (raster)
    print(infilename)
    outraster = outDir+ 'clip_'+ infilename
    print(outraster)
    result = gdal.Warp(outraster, raster, cutlineDSName = shpin, cropToCutline=True) 

#### 2. Reprojecting

In [ ]:
# Reprojecting clipped daily MODIS Geotiffs from default sinusoidal projection to desired projection (WGS 1984 UTM 12N shown)
# This code creates a new directory with reprojected clipped geotiffs

#Loading packages for this step
import gdal
import fnmatch
import os
gdal.UseExceptions()

#Directories (update this)
inDir = "/Path/to/folder/with/clipped/geotiffs"
outDir = "/Path/to/reprojected/output/folder"
os.chdir (inDir)

#Defining findRasters function (no need if done in previous step)
def findRasters (path, filter):
    for root, dirs, files in os.walk(path, filter):
        for file in fnmatch.filter(files, filter):
            yield os.path.join (root, file)
            
#Creating new rasters in outDir directory
for raster in findRasters (inDir, '*.tif'):
    (infilepath, infilename)= os.path.split (raster)
    print(infilename)
    outraster = outDir+ 'reprj_'+ infilename
    print(outraster)
    # Edit below for desired projection
    result = gdal.Warp(outraster, raster, srcSRS = '+proj=sinu +lon_0=0 +x_0=0 +y_0=0 +a=6371007.181 +b=6371007.181 +units=m +no_defs',
                   dstSRS = '+proj=utm +zone=12 +datum=WGS84 +units=m +no_defs') 

#### 3. Creating binary mask rasters

In [ ]:
#Creating snow cover fraction mask rasters for each day
#Creates daily binary rasters where values greater than the snow cover fraction threshold = 1, and others = 0. 

#Loading packages for this step
import gdal
import re
import os
import fnmatch
from matplotlib import pyplot
import numpy as np

#Directories (update this)
inDirscf = "/Path/to/folder/with/clipped/reprojected/snow/cover/geotiffs"
outDirscf = "/Path/to/output/snow/cover/mask/folder/"

#Defining functions
def findRasters (path, filter):
    for root, dirs, files in os.walk(path, filter):
        for file in fnmatch.filter(files, filter):
            yield os.path.join (root, file)

def CreateGeoTiff(outRaster, data, geo_transform, projection):
    driver = gdal.GetDriverByName('GTiff')
    rows, cols = data.shape
    DataSet = driver.Create(outRaster, cols, rows, 1, gdal.GDT_Byte)
    DataSet.SetGeoTransform(geo_transform)
    DataSet.SetProjection(projection)
    DataSet.GetRasterBand(1).WriteArray(data)
    DataSet.GetRasterBand(1).SetNoDataValue(0)
    DataSet = None
    
#Creating mask rasters   
for raster in findRasters (inDirscf, '*.tif'):
    (infilepath, infilename)= os.path.split (raster)
    print(infilename)
    scf = gdal.Open(raster, gdal.GA_ReadOnly)
    gt = scf.GetGeoTransform()
    proj = scf.GetProjection()
    scf_band = scf.GetRasterBand(1)
    scf_data = scf_band.ReadAsArray()
    #Edit fSCA values for desired snow cover fraction mask
    scf_mask = np.where(np.logical_and(scf_data>=95, scf_data<=100)==True, 1, 0)
    scf_mask_raster = outDirscf+ infilename
    data = scf_mask
    geo_transform = gt
    projection = proj
    CreateGeoTiff(scf_mask_raster, data, geo_transform, projection)

#### 4. Creating dataset

In [ ]:
#Creating xarray dataset for all clipped and reprojected RF, fSCA, and mask rasters 

#Loading packages for this step
import pandas as pd
import xarray as xr
import dask
from datetime import datetime
import rioxarray

#Directories (update this)
mask_files = "/Path/to/folder/with/mask/rasters/*.tif"
scf_values = "/Path/to/folder/with/clipped/and/reprojected/snow/cover/rasters/*.tif"
rf_values = "/Path/to/folder/with/clipped/and/reprojected/RF/rasters/*.tif"

#Defining function to add time dimension to dataset
def add_time_dim(xda):
    xda = xda.expand_dims(time = [datetime.now()])
    return xda

#Adding RF data 
rf = xr.open_mfdataset(rf_values, preprocess = add_time_dim)

#Adding dates (Update for dates of timeseries)
dates = pd.date_range('2000-02-01', '2023-09-30', freq='D')
yrs = []
yrs.append(dates)                          
t_range = yrs[0].union_many(yrs[1:]) 
rf['time'] = t_range

#Adding mask data 
mask = xr.open_mfdataset(mask_files, preprocess = add_time_dim)
mask['time'] = t_range

#Adding fSCA data 
scf_val = xr.open_mfdataset(mask_values, preprocess = add_time_dim)
scf_val['time'] = t_range

#Assigning variables to dataset 
data = rf.assign(rf = rf['band_data'], mask=mask['band_data'], scf=scf_val['band_data'])

#### 5. Snow cover fraction mask

In [ ]:
#Snow cover fraction mask step

#Adding 1 to retain RF values of 0 through mask calculation step
data['rf'] = data['rf']+1
#Mask calculation
data['rf_masked'] = data['rf'] * data['mask']
#Removing masked out data
data['rf_masked'] = data['rf_masked'].where(data['rf_masked'] != 0)
#Subtraction 1 to revert back to true RF values
data['rf_masked'] = data['rf_masked']-1

#### 6. Snow persistence mask

In [ ]:
#Snow persistence mask step

#Selecting only April-June (This is the "snowmelt season" - see paper)
def is_amj(month):
    return (month >= 4) & (month <= 6)
data['rf_daily_AMJ'] = data['rf_masked'].sel(time=is_amj(data['time.month']))

#Creating pixelcount variable: if there is data in a pixel- replace with 1, if not = replace with 0
data['pixelcount'] = data['rf_daily_AMJ'].where(data['rf_daily_AMJ']>=0, 0)
data['pixelcount'] = data['pixelcount'].where(data['pixelcount']==0, 1)

#Creating variable showing average percent (from 0 to 1) that a given pixel is included over the timeframe
data['average_pixelcount'] = data['pixelcount'].mean(dim = 'time', skipna = True)

#Creating single snow persistence mask where snow persistence >= 5% - replaced with 1, otherwise - replaced with 0
data['pixelmask'] = data['average_pixelcount'].where(data['average_pixelcount']>=.05, 0)
data['pixelmask'] = data['pixelmask'].where(data['pixelmask']<.05, 1)

#Multiplying RF data by snow persistence mask (making sure to include 0s by adding 1 to the data, and removing 1 again after the masking)
data['rf_daily_AMJ'] = data['rf_daily_AMJ']+1
data['rf_final'] = data['pixelmask'] * data['rf_daily_AMJ']
data['rf_final'] = data['rf_final'].where(data['rf_final'] != 0)
data['rf_final'] = data['rf_final']-1

#data['rf_final'] is the fully processed RF data quantified in the paper. 